In [3]:
from ultralytics import YOLO
import torch
from ultralytics.nn.autobackend import AutoBackend

In [2]:
# # Load the YOLOv8 model
# model = YOLO("yolov8s.pt")  

# # Train the model
# model.train(
#     data="datasets/data.yaml",
#     epochs=2,
#     imgsz=640,
#     batch=4,
#     name="yolov8_mask_detection",
#     project="mask-detection-project"
# )


Ultralytics 8.3.52  Python-3.10.16 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=datasets/data.yaml, epochs=2, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=mask-detection-project, name=yolov8_mask_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

train: Scanning E:\Face-mask-detection\datasets\train\labels.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]
val: Scanning E:\Face-mask-detection\datasets\val\labels.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]


Plotting labels to mask-detection-project\yolov8_mask_detection\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to mask-detection-project\yolov8_mask_detection
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G      1.486      1.692      1.321         11        640: 100%|██████████| 171/171 [22:02<00:00,  7.73s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:51<00:00,  4.68s/it]

                   all         85        319      0.652      0.533      0.567      0.365


...

        2/2         0G      1.214     0.9769      1.105         10        640: 100%|██████████| 171/171 [18:37<00:00,  6.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:34<00:00,  3.14s/it]


                   all         85        319      0.721      0.745      0.746      0.496

2 epochs completed in 0.708 hours.
Optimizer stripped from mask-detection-project\yolov8_mask_detection\weights\last.pt, 22.5MB
Optimizer stripped from mask-detection-project\yolov8_mask_detection\weights\best.pt, 22.5MB

Validating mask-detection-project\yolov8_mask_detection\weights\best.pt...
Ultralytics 8.3.52  Python-3.10.16 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:34<00:00,  3.15s/it]


                   all         85        319      0.721      0.745      0.746      0.496
             with_mask         75        260      0.746      0.935       0.93      0.624
          without_mask         30         54      0.683      0.741      0.803      0.501
 mask_weared_incorrect          4          5      0.733      0.559      0.506      0.363
Speed: 3.3ms preprocess, 368.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to mask-detection-project\yolov8_mask_detection


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002406ABA7EB0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [5]:
model_weights = "mask-detection-project/yolov8_mask_detection/weights/best.pt"
old_model_weights = "trained_weights/exp1_yolov8n_trained.pt"
test_img = "datasets/test/images/maksssksksss25.png"

In [6]:
# Load the trained YOLOv8 model
model = YOLO(model_weights)

# Run inference on an image

results = model(test_img, save=True)

# Display the results
print(results)



image 1/1 e:\Face-mask-detection\datasets\test\images\maksssksksss25.png: 640x480 1 with_mask, 663.6ms
Speed: 39.8ms preprocess, 663.6ms inference, 53.4ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict2
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'with_mask', 1: 'without_mask', 2: 'mask_weared_incorrect'}
obb: None
orig_img: array([[[175, 185, 183],
        [174, 184, 182],
        [175, 185, 182],
        ...,
        [175, 185, 177],
        [178, 188, 180],
        [181, 190, 183]],

       [[175, 185, 182],
        [174, 184, 181],
        [175, 184, 182],
        ...,
        [176, 186, 178],
        [178, 188, 180],
        [179, 188, 181]],

       [[174, 184, 182],
        [173, 183, 181],
        [174, 183, 181],
        ...,
        [178, 188, 180],
        [178, 187, 180],
        [177, 187, 179]],

       ...,

       [[129, 133, 136]

Quantisation Code

In [7]:
import torch
from ultralytics.nn.autobackend import AutoBackend

# Load the YOLOv8 model
model = AutoBackend(model_weights)  # Path to pre-trained model
model.eval()

# Dynamically quantize the model (only layers with supported types)
quantized_model = torch.quantization.quantize_dynamic(
    model.model, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8
)

# Save both the quantized architecture and weights
quantized_checkpoint = {
    "model": quantized_model,  # The quantized model itself
    "metadata": {
        "classes": model.names,  # Preserve class information for inference
    }
}

qunatised_model_weights = "yolov8_mask_detection_quantized.pt"

torch.save(quantized_checkpoint, qunatised_model_weights)
print("Quantized model saved.")


Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
Quantized model saved.


In [49]:
# Static quantisation
import torch
import torchvision.transforms as T
from ultralytics import YOLO
from PIL import Image
import cv2

test_imgs = "datasets/test/images/maksssksksss7.png"

# Load and preprocess the image
def load_image(img_path):
    img = cv2.imread(img_path)  # OpenCV loads image as BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = Image.fromarray(img)  # Convert to PIL Image
    return img

# Image preprocessing: convert image to tensor and normalize
transform = T.Compose([
    T.ToTensor(),
    T.Resize((640, 640)),  # Resize to the size expected by the model (can vary)
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Use the mean and std for imagenet, adapt if needed
])

# Load the model checkpoint
checkpoint = torch.load(model_weights, map_location='cpu')
if isinstance(checkpoint["model"], torch.nn.Module):
    model = checkpoint["model"]
else:
    model = YOLO(model_weights)
    model.load_state_dict(checkpoint["model"])

model.eval()  # Set the model to evaluation mode

# Define quantization configuration
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare model for quantization
torch.quantization.prepare(model, inplace=True)

# # Simulate data inference for calibration (for static quantization)
# for img_path in test_img:
#     img = load_image(img_path)  # Load the image
#     img = transform(img).unsqueeze(0)  # Convert to tensor and add batch dimension (unsqueeze(0))
    
#     # Convert the input tensor to Float16 to match the model weights
#     img = img.half()  # Convert tensor to half precision (float16)
    
#     # Run the model for calibration (ensure tensor input is in the same precision)
#     model(img)  # Forward pass with tensor input

# After calibration, convert the model to quantized version
quantized_model = torch.quantization.convert(model, inplace=True)

# Save the quantized model
quantized_static_model_weights = 'yolov8_mask_detection_quantized_static.pt'
torch.save(quantized_model.state_dict(), quantized_static_model_weights)

print("Quantized model saved successfully.")


e:\Face-mask-detection\.venv\lib\site-packages\torch\ao\quantization\observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
e:\Face-mask-detection\.venv\lib\site-packages\torch\ao\quantization\observer.py:1315: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


Quantized model saved successfully.


In [6]:
quantized_static_model_weights = "quantized_static_model.pt"

In [ ]:
import cv2

test_img_path = "runs\detect\predict7\maksssksksss7.jpg"
img = cv2.imread(test_img_path)

if img is None:
    print("Failed to read the image using OpenCV.")
else:
    print("Image loaded successfully.")
    # Show the image to verify
    cv2.imshow("Test Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Image loaded successfully.


In [7]:
import torch

# Load the quantized YOLOv8 model
checkpoint = torch.load(quantized_static_model_size, map_location="cpu")

# Retrieve the model and metadata
quantized_model = checkpoint["model"]
metadata = checkpoint["metadata"]

# Ensure the model is in evaluation mode
quantized_model.eval()

# Perform inference
import time

# Dummy test input
test_img = torch.randn(1, 3, 640, 640)

# Measure latency
start_time = time.time()
with torch.no_grad():
    results = quantized_model(test_img)
end_time = time.time()

latency = (end_time - start_time) * 1000  # Convert seconds to milliseconds
print(f"Inference Latency: {latency:.2f} ms")


AttributeError: 'float' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [46]:
# Load the checkpoint
checkpoint = torch.load(qunatised_model_weights, map_location="cpu")

# Check the keys to find the model
print("Checkpoint Keys:", checkpoint.keys())

# If the model state_dict is in a different key, replace 'model' with the correct one
if 'model' in checkpoint:
    quantized_model = checkpoint['model']
elif 'model_state_dict' in checkpoint:
    quantized_model = checkpoint['model_state_dict']
else:
    raise KeyError("Model not found in checkpoint")

# Ensure the model is in evaluation mode
quantized_model.eval()


Checkpoint Keys: dict_keys(['model', 'metadata'])


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): ModuleList(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
        )
      )
    )
    (3): Conv(
      (conv): Conv2d(64

In [62]:
import torch
import torchvision.transforms as T
from ultralytics import YOLO
from PIL import Image
import cv2
import pickle
import gc
from threading import Lock

# Paths
model_weights = "mask-detection-project/yolov8_mask_detection/weights/best.pt"
quantized_model_weights_path = "quantized_static_model_state_dict.pth"
test_img_path = "datasets/test/images/maksssksksss7.png"

# Function: Load and preprocess the image
def load_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

# Preprocessing pipeline
transform = T.Compose([
    T.ToTensor(),
    T.Resize((640, 640)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function: Remove non-serializable attributes
def remove_non_serializable_attributes(obj):
    for attr in dir(obj):
        try:
            pickle.dumps(getattr(obj, attr))  # Test if attribute is serializable
        except (pickle.PickleError, TypeError):
            print(f"Removing non-serializable attribute: {attr}")
            delattr(obj, attr)  # Remove problematic attribute
    return obj

# Function: Recursively remove thread locks
def clean_thread_locks(obj):
    for ref in gc.get_referrers(obj):
        if isinstance(ref, dict):
            for key, value in list(ref.items()):
                if isinstance(value, Lock):
                    del ref[key]
    return obj

# Step 1: Load YOLO model
model = YOLO(model_weights)
model.eval()  # Set model to evaluation mode

# Step 2: Prepare the model for quantization
try:
    model.fuse()  # Fuse layers (if applicable)
    model.qconfig = torch.quantization.get_default_qconfig("fbgemm")
    torch.quantization.prepare(model, inplace=True)
    print("Model prepared for quantization.")
except Exception as e:
    print("Quantization preparation skipped due to:", e)

# Step 3: Calibrate the model using an image
img = transform(load_image(test_img_path)).unsqueeze(0).div(255)
with torch.no_grad():
    model(img)

# Step 4: Convert the model to a quantized version
try:
    quantized_model = torch.quantization.convert(model, inplace=False)
    print("Model successfully quantized.")
except Exception as e:
    print("Quantization failed:", e)
    quantized_model = model

# Step 5: Attempt to clean and save the quantized model
try:
    # Remove non-serializable attributes
    quantized_model = remove_non_serializable_attributes(quantized_model)
    quantized_model = clean_thread_locks(quantized_model)

    # Save the quantized model as state_dict
    torch.save(quantized_model.state_dict(), quantized_model_weights_path)
    print("Quantized model state_dict saved successfully.")
except Exception as e:
    print("Error during quantized model saving:", e)

# Step 6: Reload the saved quantized model and validate inference
try:
    # Rebuild model and load state_dict
    reloaded_model = YOLO(model_weights)  # Reload architecture
    reloaded_model.load_state_dict(torch.load(quantized_model_weights_path))
    reloaded_model.eval()
    print("Reloaded model loaded successfully.")

    # Perform inference
    img = transform(load_image(test_img_path)).unsqueeze(0).div(255)
    with torch.no_grad():
        output = reloaded_model(img)
    print("Inference output:", output)
except Exception as e:
    print("Error during reloading or inference:", e)

# Step 7: Export to ONNX as a fallback (if needed)
try:
    dummy_input = torch.randn(1, 3, 640, 640)  # Match input shape
    torch.onnx.export(quantized_model, dummy_input, "model_quantized.onnx", opset_version=12)
    print("Model exported to ONNX successfully.")
except Exception as e:
    print("ONNX export failed:", e)


Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
Model prepared for quantization.

0: 640x640 (no detections), 750.4ms
Speed: 0.0ms preprocess, 750.4ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
Quantization failed: cannot pickle '_thread.lock' object
Removing non-serializable attribute: T_destination
Error during quantized model saving: T_destination
Error during reloading or inference: [Errno 2] No such file or directory: 'quantized_static_model_state_dict.pth'
Ultralytics 8.3.52  Python-3.10.16 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
engine\trainer: task=detect, mode=train, model=mask-detection-project/yolov8_mask_detection/weights/best.pt, data=datasets/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=Fal

train: Scanning E:\Face-mask-detection\datasets\train\labels.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]
val: Scanning E:\Face-mask-detection\datasets\val\labels.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/43 [00:26<?, ?it/s]


KeyboardInterrupt: 

Latency code

In [42]:
import cv2
import torch
import time
import numpy as np

# Load test image and preprocess
test_img_path = "datasets/test/images/maksssksksss7.png"
image = cv2.imread(test_img_path)  # Read image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
image = cv2.resize(image, (640, 640))  # Resize to YOLOv8 input size
image = image / 255.0  # Normalize pixel values to [0, 1]
image = np.transpose(image, (2, 0, 1))  # Change HWC to CHW format
image = np.expand_dims(image, axis=0)  # Add batch dimension
image_tensor = torch.tensor(image, dtype=torch.float32)  # Convert to tensor

# Measure latency
start_time = time.time()
with torch.no_grad():
    results = quantized_model(image_tensor)
end_time = time.time()

latency = (end_time - start_time) * 1000  # Milliseconds
print(f"Inference Latency: {latency:.2f} ms")


Inference Latency: 436.12 ms


In [13]:
import cv2
import torch
import time
import numpy as np

# Load test image and preprocess
test_img_path = "datasets/test/images/maksssksksss7.png"
image = cv2.imread(test_img_path)  # Read image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
image = cv2.resize(image, (640, 640))  # Resize to YOLOv8 input size
image = image / 255.0  # Normalize pixel values to [0, 1]
image = np.transpose(image, (2, 0, 1))  # Change HWC to CHW format
image = np.expand_dims(image, axis=0)  # Add batch dimension
image_tensor = torch.tensor(image, dtype=torch.float32)  # Convert to tensor

# Measure latency
start_time = time.time()
with torch.no_grad():
    results = model(image_tensor)
end_time = time.time()

latency = (end_time - start_time) * 1000  # Milliseconds
print(f"Inference Latency: {latency:.2f} ms")


Inference Latency: 1028.34 ms


FPS (Frames per Second)

In [43]:
import torch
import cv2
import time
import numpy as np
from torchvision import transforms

# Define the image transformation (resize and normalize)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((640, 640)),  # Resize to the input size expected by YOLO
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Typical normalization values
])

# Function to load image and preprocess
def load_image(img_path):
    img = cv2.imread(img_path)  # Load the image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    return img

# List of test image paths (ensure this list contains valid paths to your test images)
test_img = [
    "datasets/test/images/maksssksksss7.png",
    "datasets/test/images/maksssksksss25.png",
    "datasets/test/images/maksssksksss36.png",
    # Add more image paths as required
]

# Start FPS calculation
fps_start_time = time.time()
num_frames = len(test_img)  # Number of images to process

for img_path in test_img:
    img = load_image(img_path)  # Load each image
    img = transform(img).unsqueeze(0)  # Apply transformations and add batch dimension (1, C, H, W)
    
    # Perform inference on each image
    with torch.no_grad():  # Disable gradient computation for inference
        quantized_model(img) 

fps_end_time = time.time()

# Calculate FPS
fps = num_frames / (fps_end_time - fps_start_time)
print(f"FPS (frames per second): {fps:.2f}")


FPS (frames per second): 1.41


In [6]:
import os

# Check the model size (in MB)
original_model_size = os.path.getsize(model_weights) / (1024 * 1024)  # Size in MB
# quantized_model_size = os.path.getsize(qunatised_model_weights) / (1024 * 1024)
quantized_static_model_size = os.path.getsize(quantized_model_weights) / (1024 * 1024)

# Model size reduction ratio
size_reduction_ratio = (original_model_size - quantized_static_model_size) / original_model_size * 100

print(f"Original model size: {original_model_size:.2f} MB")
# print(f"Quantized model size: {quantized_model_size:.2f} MB")
print(f"Quantized Static model size: {quantized_static_model_size:.2f} MB")
print(f"Model size reduction: {size_reduction_ratio:.2f}%")


Original model size: 21.45 MB
Quantized Static model size: 11.00 MB
Model size reduction: 48.71%


In [20]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import time
from utils.metrics import ap_per_class, ConfusionMatrix

# Load the trained and quantized YOLOv5 model (use 'yolov5_quantized.pt' for quantized version)
model = torch.load(qunatised_model_weights)  # Load model into memory
model.eval()  # Set to eval mode

# Load your test dataset or image here (use for batch inference)
# Example: Load images for evaluation from a directory
test_images = 'datasets/test/images/'  # Path to your test dataset

# Confusion matrix for accuracy calculation
conf_matrix = ConfusionMatrix()

# Tracking precision, recall, and F1 score
y_true = []  # Ground truth labels
y_pred = []  # Model predictions

# Time the inference and track FPS
start_time = time.time()
for img_path in test_images:  # Loop through images
    # Load the image (implement your custom data loader here)
    img = load_image(img_path)
    results = model(img)
    
    # Get the predictions (labels predicted and the confidence)
    pred_labels = results.pred[0][:, -1]  # Class predictions (e.g., face mask or no mask)
    
    # Ground truth labels (true face mask presence in test dataset)
    gt_labels = get_ground_truth_labels(img_path)  # You need to implement this function
    
    # Add to confusion matrix
    conf_matrix.process(gt_labels, pred_labels)
    
    # Save predictions and truth labels for precision, recall calculation
    y_true.extend(gt_labels)
    y_pred.extend(pred_labels)

# Evaluate metrics
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Calculate inference time
end_time = time.time()
inference_time = end_time - start_time

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Inference time per image: {inference_time/len(test_images):.2f} seconds")


ModuleNotFoundError: No module named 'utils'

In [1]:
import torch
import torchvision.transforms as T
from ultralytics import YOLO
from PIL import Image
import cv2

# Paths
model_weights = "mask-detection-project/yolov8_mask_detection/weights/best.pt"
quantized_model_weights_path = "quantized_dynamic_model_state_dict.pth"
test_img_path = "datasets/test/images/maksssksksss7.png"

# Function: Load and preprocess the image
def load_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

# Preprocessing pipeline
transform = T.Compose([
    T.ToTensor(),
    T.Resize((640, 640)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Step 1: Load YOLO model
model = YOLO(model_weights)
model.eval()  # Set to evaluation mode

# Step 2: Apply dynamic quantization
try:
    # Quantize specific parts of the model such as Linear layers
    quantized_model = torch.quantization.quantize_dynamic(
        model,  # Target model
        {torch.nn.Linear},  # Specify the layer types to quantize
        dtype=torch.qint8  # Target dtype
    )
    print("Dynamic quantization applied successfully.")
except Exception as e:
    print("Error during dynamic quantization:", e)
    quantized_model = model  # Fallback to original model if quantization fails

# Step 3: Save the quantized model state_dict
try:
    torch.save(quantized_model.state_dict(), quantized_model_weights_path)
    print("Quantized model state_dict saved successfully.")
except Exception as e:
    print("Error saving quantized model:", e)

# Step 4: Reload the saved quantized model and validate inference
try:
    # Rebuild model and load state_dict
    reloaded_model = YOLO(model_weights)  # Reload architecture
    reloaded_model.load_state_dict(torch.load(quantized_model_weights_path))
    reloaded_model.eval()
    print("Reloaded model loaded successfully.")

    # Perform inference
    img = transform(load_image(test_img_path)).unsqueeze(0).div(255)
    with torch.no_grad():
        output = reloaded_model(img)
    print("Inference output:", output)
except Exception as e:
    print("Error during reloading or inference:", e)


Dynamic quantization applied successfully.
Quantized model state_dict saved successfully.
Reloaded model loaded successfully.

0: 640x640 (no detections), 1566.2ms
Speed: 27.8ms preprocess, 1566.2ms inference, 116.0ms postprocess per image at shape (1, 3, 640, 640)
Inference output: [ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'with_mask', 1: 'without_mask', 2: 'mask_weared_incorrect'}
obb: None
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 1, 1],
        [0, 1, 1],
        [0, 1, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 1, 1],
        [0, 1, 1],
        [0, 1, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],


In [3]:
import torch.nn.utils.prune as prune

# Apply pruning
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name="weight", amount=0.3)  # Prune 30% weights

# Remove pruning masks after applying
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.remove(module, "weight")


In [12]:
import time

# Benchmark original and quantized models
def benchmark_model(model, input_tensor):
    model.eval()
    with torch.no_grad():
        start_time = time.time()
        for _ in range(100):  # Repeat for consistent results
            _ = model(input_tensor)
        end_time = time.time()
    return (end_time - start_time) / 100

# Create a dummy input
dummy_input = torch.randn(1, 3, 640, 640)
text_input = "datasets/test/images/"

original_time = benchmark_model(model, dummy_input)
quantized_time = benchmark_model(quantized_model, dummy_input)

print(f"Original Model Avg Time: {original_time:.4f} seconds")
print(f"Quantized Model Avg Time: {quantized_time:.4f} seconds")


Original Model Avg Time: 0.4297 seconds
Quantized Model Avg Time: 0.4042 seconds


In [6]:
import time

# Benchmark original and quantized models
def benchmark_model(model, input_tensor):
    model.eval()
    with torch.no_grad():
        start_time = time.time()
        # for _ in range(100):  # Repeat for consistent results
        _ = model(input_tensor)
        end_time = time.time()
    return (end_time - start_time) / 100

# Create a dummy input
dummy_input = torch.randn(1, 3, 640, 640)
text_input = "datasets/test/images/"

original_time = benchmark_model(model, text_input)
quantized_time = benchmark_model(quantized_model, dummy_input)

print(f"Original Model Avg Time: {original_time:.4f} seconds")
print(f"Quantized Model Avg Time: {quantized_time:.4f} seconds")


image 1/86 e:\Face-mask-detection\datasets\test\images\maksssksksss107.png: 640x512 1 with_mask, 352.0ms
image 2/86 e:\Face-mask-detection\datasets\test\images\maksssksksss119.png: 480x640 5 with_masks, 2 without_masks, 293.0ms
image 3/86 e:\Face-mask-detection\datasets\test\images\maksssksksss126.png: 640x448 2 with_masks, 1 without_mask, 302.2ms
image 4/86 e:\Face-mask-detection\datasets\test\images\maksssksksss133.png: 640x512 1 with_mask, 348.0ms
image 5/86 e:\Face-mask-detection\datasets\test\images\maksssksksss142.png: 480x640 1 with_mask, 316.9ms
image 6/86 e:\Face-mask-detection\datasets\test\images\maksssksksss147.png: 416x640 3 with_masks, 266.1ms
image 7/86 e:\Face-mask-detection\datasets\test\images\maksssksksss155.png: 384x640 6 with_masks, 263.6ms
image 8/86 e:\Face-mask-detection\datasets\test\images\maksssksksss157.png: 384x640 7 with_masks, 1 without_mask, 246.4ms
image 9/86 e:\Face-mask-detection\datasets\test\images\maksssksksss158.png: 448x640 15 with_masks, 529.3ms

In [4]:
import torch
from ultralytics.nn.autobackend import AutoBackend

# Load YOLOv8 model
model = AutoBackend(model_weights)  # Path to pre-trained model
model.eval()

# Set quantization config
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare model for static quantization (this step prepares the model for calibration)
torch.quantization.prepare(model, inplace=True)

# Simulate calibration with a dummy input (calibration step needed to gather statistics)
dummy_input = torch.randn(1, 3, 640, 640)  # Adjust to model input shape
with torch.no_grad():
    for _ in range(100):  # Simulate calibration steps
        model(dummy_input)

# Apply static quantization after calibration
quantized_model = torch.quantization.convert(model, inplace=False)

# Save the quantized model checkpoint
quantized_checkpoint = {
    "model": quantized_model,
    "metadata": {
        "classes": model.names  # Preserve class information for inference
    }
}

quantized_model_weights = "yolov8_mask_detection_static_quantized.pt"
torch.save(quantized_checkpoint, quantized_model_weights)

print("Static quantized model saved.")


Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


e:\Face-mask-detection\.venv\lib\site-packages\torch\ao\quantization\observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Static quantized model saved.


In [ ]:
import torch
from PIL import Image, ImageDraw
import torchvision.transforms as T
import time
import os
from torchvision.ops import nms  # Non-Maximum Suppression

# Load the quantized model
quantized_model_path = "yolov8_mask_detection_quantized.pt"
checkpoint = torch.load(quantized_model_path)  # Load checkpoint
quantized_model = checkpoint['model']  # Extract the actual model

quantized_model.eval()  # Set to evaluation mode

# Preprocessing transformation (based on YOLOv8 requirements)
transform = T.Compose([T.Resize((640, 640)),
                       T.ToTensor(),
                       T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Load and preprocess the image
image_path = 'datasets/test/images/maksssksksss7.png'  # Sample path, change as required
img = Image.open(image_path)

# Save the original image size (before transformation)
original_size = img.size  # (width, height)

# Convert RGBA to RGB if the image has an alpha channel
if img.mode == 'RGBA':
    img = img.convert('RGB')

# Apply transformation to the image
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

# Measure inference time for the quantized model
start_time = time.time()
with torch.no_grad():
    # Pass the image through the model for inference
    predictions = quantized_model(img_tensor)  # Make predictions
quantized_inference_time = time.time() - start_time
print(f"Quantized Model Inference Time: {quantized_inference_time:.4f} seconds")

# Post-process the predictions to get boxes and labels
predictions = predictions[0]  # Get batch 0 since batch_size=1

# If no detections, return empty boxes and labels
if predictions.size(0) == 0:
    print("No predictions found")
    boxes, scores, labels = torch.tensor([]), torch.tensor([]), torch.tensor([])
else:
    # Extract boxes, scores, and labels
    boxes = predictions[:, :4]  # Coordinates (x1, y1, x2, y2)
    scores = predictions[:, 4]  # Confidence score
    labels = predictions[:, 5]  # Class labels

    # Reshape boxes, scores, and labels to 2D tensor (num_boxes, 4), (num_boxes,), and (num_boxes,)
    boxes = boxes.view(-1, 4)  # Flatten the boxes tensor
    scores = scores.view(-1)  # Flatten the scores tensor
    labels = labels.view(-1)  # Flatten the labels tensor

    # Apply Non-Maximum Suppression (NMS) to filter boxes based on score and overlap
    nms_indices = nms(boxes, scores, 0.5)  # 0.5 is the IoU threshold for NMS

    # Filter boxes, scores, and labels based on NMS indices
    boxes = boxes[nms_indices]
    scores = scores[nms_indices]
    labels = labels[nms_indices]

    # Scale the bounding box coordinates back to the original image size
    scale_x = original_size[0] / 640  # Scale factor for width
    scale_y = original_size[1] / 640  # Scale factor for height

    # Scale the boxes to match the original image dimensions
    boxes = boxes.clone()  # Avoid in-place changes
    boxes[:, [0, 2]] *= scale_x  # Scale x1 and x2
    boxes[:, [1, 3]] *= scale_y  # Scale y1 and y2

    # Ensure the coordinates are properly scaled and stay within image bounds
    boxes = boxes.clamp(min=0)  # Clamp to avoid negative values or out-of-bound coordinates

# Check if valid boxes exist and draw on the image
draw = ImageDraw.Draw(img)

if boxes.size(0) > 0:  # Check if there are valid predictions
    for i, box in enumerate(boxes):
        score = scores[i].item()
        label = int(labels[i].item())  # Ensure label is an integer

        # Draw rectangle around the detected object
        x1, y1, x2, y2 = box.tolist()

        # Ensure coordinates stay within original image bounds
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(original_size[0], x2), min(original_size[1], y2)

        # Draw a rectangle
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

        # Add label and score text
        text = f"Label: {label}, Score: {score:.2f}"  # Customize label text as needed
        draw.text((x1, y1), text, fill="red")

        print(f"Prediction: Box({x1}, {y1}, {x2}, {y2}), Label: {label}, Score: {score:.2f}")
else:
    print("No valid boxes to draw")

# Save the image with annotations
output_dir = 'runs/detect/'

# Create the directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Save the image with the prefix "quantised"
output_image_path = os.path.join(output_dir, 'quantised_' + os.path.basename(image_path))
img.save(output_image_path)
print(f"Image with predictions saved to {output_image_path}")


Quantized Model Inference Time: 0.6175 seconds
Prediction: Box(19.07632827758789, 21.54876708984375, 41.66313171386719, 29.659194946289062), Label: 0, Score: 0.33
Prediction: Box(17.96741485595703, 27.51082420349121, 56.799163818359375, 36.390480041503906), Label: 0, Score: 0.32
Prediction: Box(12.03826904296875, 9.056410789489746, 14.77712631225586, 12.774152755737305), Label: 0, Score: 0.31
Prediction: Box(12.398223876953125, 9.544801712036133, 14.263896942138672, 12.269010543823242), Label: 0, Score: 0.27
Prediction: Box(12.003402709960938, 9.199333190917969, 12.418441772460938, 9.106893539428711), Label: 0, Score: 0.25
Prediction: Box(16.407947540283203, 14.367313385009766, 30.229644775390625, 25.608558654785156), Label: 0, Score: 0.22
Prediction: Box(13.274497985839844, 10.811676979064941, 16.159706115722656, 14.894886016845703), Label: 0, Score: 0.22
Prediction: Box(10.952320098876953, 7.642879486083984, 11.27542495727539, 8.898968696594238), Label: 0, Score: 0.20
Prediction: Box

In [77]:
import torch
from PIL import Image, ImageDraw
import torchvision.transforms as T
import time
import os
from torchvision.ops import nms  # Non-Maximum Suppression

# Load the quantized model
quantized_model_path = "yolov8_mask_detection_quantized.pt"
checkpoint = torch.load(quantized_model_path)  # Load checkpoint
quantized_model = checkpoint['model']  # Extract the actual model

quantized_model.eval()  # Set to evaluation mode

# Preprocessing transformation (based on YOLOv8 requirements)
transform = T.Compose([T.Resize((640, 640)),
                       T.ToTensor(),
                       T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Load and preprocess the image
image_path = 'datasets/test/images/maksssksksss7.png'  # Sample path, change as required
img = Image.open(image_path)

# Save the original image size (before transformation)
original_size = img.size  # (width, height)

# Convert RGBA to RGB if the image has an alpha channel
if img.mode == 'RGBA':
    img = img.convert('RGB')

# Apply transformation to the image
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

# Measure inference time for the quantized model
start_time = time.time()
with torch.no_grad():
    # Pass the image through the model for inference
    predictions = quantized_model(img_tensor)  # Make predictions
quantized_inference_time = time.time() - start_time
print(f"Quantized Model Inference Time: {quantized_inference_time:.4f} seconds")

# Post-process the predictions to get boxes and labels
predictions = predictions[0]  # Get batch 0 since batch_size=1

# If no detections, return empty boxes and labels
if predictions.size(0) == 0:
    print("No predictions found")
    boxes, scores, labels = torch.tensor([]), torch.tensor([]), torch.tensor([])
else:
    # Extract boxes, scores, and labels
    boxes = predictions[:, :4]  # Coordinates (x1, y1, x2, y2)
    scores = predictions[:, 4]  # Confidence score
    labels = predictions[:, 5]  # Class labels

    # Reshape boxes, scores, and labels to 2D tensor (num_boxes, 4), (num_boxes,), and (num_boxes,)
    boxes = boxes.view(-1, 4)  # Flatten the boxes tensor
    scores = scores.view(-1)  # Flatten the scores tensor
    labels = labels.view(-1)  # Flatten the labels tensor

    # Apply confidence threshold of 90% and above
    confidence_threshold = 0.9
    valid_indices = scores >= confidence_threshold
    boxes = boxes[valid_indices]
    scores = scores[valid_indices]
    labels = labels[valid_indices]

    # Apply Non-Maximum Suppression (NMS) to filter boxes based on score and overlap
    nms_indices = nms(boxes, scores, 0.5)  # 0.5 is the IoU threshold for NMS

    # Filter boxes, scores, and labels based on NMS indices
    boxes = boxes[nms_indices]
    scores = scores[nms_indices]
    labels = labels[nms_indices]

    # Scale the bounding box coordinates back to the original image size
    scale_x = original_size[0] / 640  # Scale factor for width
    scale_y = original_size[1] / 640  # Scale factor for height

    # Scale the boxes to match the original image dimensions
    boxes = boxes.clone()  # Avoid in-place changes
    boxes[:, [0, 2]] *= scale_x  # Scale x1 and x2
    boxes[:, [1, 3]] *= scale_y  # Scale y1 and y2

    # Ensure the coordinates are properly scaled and stay within image bounds
    boxes = boxes.clamp(min=0)  # Clamp to avoid negative values or out-of-bound coordinates

# Check if valid boxes exist and draw on the image
draw = ImageDraw.Draw(img)

if boxes.size(0) > 0:  # Check if there are valid predictions
    for i, box in enumerate(boxes):
        score = scores[i].item()
        label = int(labels[i].item())  # Ensure label is an integer

        # Draw rectangle around the detected object
        x1, y1, x2, y2 = box.tolist()

        # Ensure coordinates stay within original image bounds
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(original_size[0], x2), min(original_size[1], y2)

        # Draw a rectangle
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

        # Add label and score text
        text = f"Label: {label}, Score: {score:.2f}"  # Customize label text as needed
        draw.text((x1, y1), text, fill="red")

        print(f"Prediction: Box({x1}, {y1}, {x2}, {y2}), Label: {label}, Score: {score:.2f}")
else:
    print("No valid boxes to draw")

# Save the image with annotations
output_dir = 'runs/detect/'

# Create the directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Save the image with the prefix "quantised"
output_image_path = os.path.join(output_dir, 'quantised_' + os.path.basename(image_path))
img.save(output_image_path)
print(f"Image with predictions saved to {output_image_path}")


Quantized Model Inference Time: 0.5975 seconds
No valid boxes to draw
Image with predictions saved to runs/detect/quantised_maksssksksss7.png


In [ ]:
import torch
from ultralytics import YOLO

# Load the TorchScript model (quantized) for evaluation
quantized_model_path = "yolov8_mask_detection_static_quantized.pt"
quantized_model = torch.load(quantized_model_path)

# Preprocessing transformation (same as YOLOv8 expectations)
from torchvision import transforms as T
transform = T.Compose([T.Resize((640, 640)),
                       T.ToTensor(),
                       T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Run mAP evaluation on test dataset (similar to how YOLOv8 expects)
results = quantized_model.val(data='datasets/test/data.yaml')  # You need the data.yaml for evaluation
print(f"mAP: {results.metrics['mAP_0.5']:.4f}")

# Extract evaluation metrics
metrics = results.metrics
precision = metrics.get('precision_0.5', [])
recall = metrics.get('recall_0.5', [])
f1_score = metrics.get('f1_score_0.5', [])

print(f"Precision: {precision[-1]}")
print(f"Recall: {recall[-1]}")
print(f"F1 Score: {f1_score[-1]}")


TypeError: 'dict' object is not callable

In [1]:
# ! pip install torch-tensorrt

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.9 MB 10.0 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.9 MB 10.0 MB/s eta 0:00:01
   --------

YOLO with QAT

In [16]:
import torch
import torch.nn.functional as F
from torch.quantization import get_default_qat_qconfig, prepare_qat, convert
from ultralytics import YOLO
import torchvision.transforms as T
from PIL import Image

# Paths
model_weights = "mask-detection-project/yolov8_mask_detection/weights/best.pt"
test_img_path = "datasets/test/images/maksssksksss7.png"

# Load YOLO model and prepare for QAT
yolo_model = YOLO(model_weights)  # Load YOLOv8 model
pytorch_model = yolo_model.model  # Access underlying PyTorch model
pytorch_model.train()  # Set model to training mode

# QAT Configuration
pytorch_model.qconfig = get_default_qat_qconfig("fbgemm")
prepare_qat(pytorch_model, inplace=True)

# Data Loader for Custom Dataset
def custom_data_loader(batch_size=4):
    """
    Simulates a custom dataset with images and labels for YOLOv8 format:
    - Images: Tensor of shape (B, 3, H, W).
    - Labels: List of tensors (one per image) with shape (N, 5), where N is the number of boxes.
      Format: [class_id, x_center, y_center, width, height] (normalized).
    """
    for _ in range(10):  # Simulated 10 batches
        imgs = torch.rand(batch_size, 3, 640, 640)  # Random image data
        labels = [torch.tensor([[0, 0.5, 0.5, 0.2, 0.3]]) for _ in range(batch_size)]  # Dummy labels
        yield imgs, labels

# Custom Loss Function
def custom_loss_function(predictions, labels):
    """
    Custom loss function for YOLOv8:
    - predictions: Raw predictions from the YOLO model.
    - labels: Ground truth in YOLO format.
    """
    if isinstance(predictions, (list, tuple)):
        predictions = predictions[0]  # Unpack YOLOv8 predictions if needed

    cls_preds = predictions[..., 5:]  # Classification predictions
    box_preds = predictions[..., :4]  # Bounding box predictions

    # Ensure target tensors are moved to the correct device
    batch_size = len(labels)
    box_targets = torch.zeros_like(box_preds, device=box_preds.device)
    class_targets = torch.zeros(cls_preds.shape[:-1], dtype=torch.long, device=cls_preds.device)

    # Reformat labels into batch format
    for i, label in enumerate(labels):
        num_labels = label.size(0)
        box_targets[i, :num_labels] = label[:, 1:]
        class_targets[i, :num_labels] = label[:, 0].long()

    # Classification Loss
    cls_loss = F.cross_entropy(
        cls_preds.view(-1, cls_preds.size(-1)), 
        class_targets.view(-1)
    )

    # Bounding Box Loss
    box_loss = generalized_box_iou_loss(
        box_preds.view(-1, 4),
        box_targets.view(-1, 4)
    )

    # Log gradients
    print("Box_preds requires_grad:", box_preds.requires_grad)
    print("Cls_preds requires_grad:", cls_preds.requires_grad)
    
    return cls_loss + box_loss


# Optimizer
optimizer = torch.optim.Adam(pytorch_model.parameters(), lr=1e-4)
epochs = 10

# Training Loop
for epoch in range(epochs):
    for imgs, labels in custom_data_loader():
        optimizer.zero_grad()

        # Forward pass
        predictions = pytorch_model(imgs)
        
        # YOLOv8-specific predictions adjustment
        if isinstance(predictions, (list, tuple)):
            predictions = predictions[0]

        # Compute loss
        loss = custom_loss_function(predictions, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{epochs} completed. Loss: {loss.item()}")

# Convert Model to Quantized Version
pytorch_model.eval()
quantized_model = convert(pytorch_model, inplace=True)

# Inference Function
def inference(test_img_path, model):
    """
    Perform inference with a trained YOLOv8 quantized model.
    """
    img = T.ToTensor()(Image.open(test_img_path).convert("RGB")).unsqueeze(0).float() / 255.0
    with torch.no_grad():
        output = model(img)
    return output

# Test Inference
print("Testing inference with quantized model...")
result = inference(test_img_path, quantized_model)
print("Inference Result:", result)


Box_preds requires_grad: False
Cls_preds requires_grad: False


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [5]:
import torch
from ultralytics import YOLO

# Paths
model_weights = "yolov8s.pt"
quantized_model_weights_path = "quantized_qat_model_weights.pth"
data_yaml_path = "datasets/data.yaml"  # Path to your data.yaml file

# Load YOLO model
model = YOLO(model_weights)

# Fuse the model (optional for optimization)
model.fuse()  # Fuses Conv2d + BatchNorm layers

epochs = 1
batch_size = 4
data_yaml = "datasets/data.yaml"  # Ensure this is correctly set up with paths to images and annotations.

# # Start Training
# model.train(
#     data=data_yaml,
#     epochs=epochs,
#     batch=batch_size,
#     imgsz=640,
#     project="qat_yolov8_project",
#     name="face_mask_detection"
# )
model.qconfig = torch.quantization.get_default_qat_qconfig("fbgemm")

# Prepare for QAT
torch.quantization.prepare_qat(model.train(
    data=data_yaml,
    epochs=epochs,
    batch=batch_size,
    imgsz=640,
    project="qat_yolov8_project",
    name="face_mask_detection"
), inplace=True)

# Train the model with QAT
model.train(data=data_yaml_path, epochs=1, batch=4, imgsz=640, device=0)

# Convert to a quantized model
model.eval()
quantized_model = torch.quantization.convert(model, inplace=True)

# Save the quantized QAT model
torch.save(quantized_model.state_dict(), quantized_model_weights_path)
print("Quantized QAT model saved successfully.")

# Load and test quantized model
def load_quantized_model():
    reloaded_model = YOLO(model_weights)  # Reinstantiate YOLO structure
    reloaded_model.load_state_dict(torch.load(quantized_model_weights_path, map_location="cpu"))
    reloaded_model.eval()
    return reloaded_model

quantized_model_reloaded = load_quantized_model()

# Inference
img = torch.rand(1, 3, 640, 640)  # Replace with actual image tensor
with torch.no_grad():
    output = quantized_model_reloaded(img)
print("Inference output:", output)


YOLOv8s summary (fused): 168 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
Ultralytics 8.3.52  Python-3.10.16 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=datasets/data.yaml, epochs=1, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=qat_yolov8_project, name=face_mask_detection2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning E:\Face-mask-detection\datasets\train\labels.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]
val: Scanning E:\Face-mask-detection\datasets\val\labels.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]


Plotting labels to qat_yolov8_project\face_mask_detection2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to qat_yolov8_project\face_mask_detection2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.894      2.674      1.588         11        640: 100%|██████████| 171/171 [19:08<00:00,  6.71s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:38<00:00,  3.53s/it]


                   all         85        319      0.867      0.331      0.402      0.258

1 epochs completed in 0.333 hours.
Optimizer stripped from qat_yolov8_project\face_mask_detection2\weights\last.pt, 22.5MB
Optimizer stripped from qat_yolov8_project\face_mask_detection2\weights\best.pt, 22.5MB

Validating qat_yolov8_project\face_mask_detection2\weights\best.pt...
Ultralytics 8.3.52  Python-3.10.16 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:31<00:00,  2.86s/it]


                   all         85        319      0.866      0.333      0.402      0.259
             with_mask         75        260      0.801      0.781      0.822      0.542
          without_mask         30         54      0.796      0.217      0.348      0.204
 mask_weared_incorrect          4          5          1          0     0.0368     0.0294
Speed: 2.7ms preprocess, 321.9ms inference, 0.0ms loss, 20.2ms postprocess per image
Results saved to qat_yolov8_project\face_mask_detection2


AttributeError: 'DetMetrics' object has no attribute 'training'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP) of an
    object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (dict of str): A dict of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass plots path and data whe